# Part 1: Scrape, data cleaning and creating data frame

### Import the required libraries

In [1]:
import requests
import lxml.html as lh
import pandas as pd

### Scrape the wikisite to put the table in this notebook

In [2]:
canadapost_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #assign the wiki page

page = requests.get(canadapost_url) # create a handle to for contents of the wiki page

doc = lh.fromstring(page.content) # store content of the wiki page under doc

tr_elements = doc.xpath('//tr') # parse data stored between tr in the html

[len(T) for T in tr_elements[:12]] # check the length of the first 12 rows

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

### Parse the table header in the first row

In [3]:
tr_elements = doc.xpath('//tr') # parse first row as header

col = [] # create empty list
i = 0

for t in tr_elements[0]: # for each row, store each first element (header) and an empty list
    i+=1
    name=t.text_content()
    print("%d:%s" % (i,name))
    col.append((name,[]))

1:Postcode
2:Borough
3:Neighbourhood



### Parse the data in the subsequent rows

In [4]:
for j in range(1,len(tr_elements)): # Because header is the first row, data would be store in the subsequent rows.
    T = tr_elements[j] #T is j'th row
    
    if len(T)!=3: #if row is not size 3, //tr data is not from the table.
        break
        
    i = 0 #i is the index of the first column
    
    for t in T.iterchildren(): #iterate through each element of the row
        data=t.text_content()
            
        col[i][1].append(data) #append the data to the empty list of the i'th column
            
        i+=1 #increment i for the next column

### Check the number of rows in a column. There are 287 rows and 3 columns

In [5]:
[len(C) for (title,C) in col]

[287, 287, 287]

### Create and print the dataframe

In [6]:
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)

In [7]:
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M6A,North York,Lawrence Heights\n
6,M6A,North York,Lawrence Manor\n
7,M7A,Downtown Toronto,Queen's Park\n
8,M8A,Not assigned,Not assigned\n
9,M9A,Queen's Park,Not assigned\n


### See how many rows and columns in the above dataframe. It should be 287 columns and 3 rows.

In [8]:
df.shape

(287, 3)

### There seem to be the \n in the Neighbourhood column. It is best  to remove it so the dataframe looks neat.

In [9]:
df = df.replace('\n','', regex=True) #this is to remove \n
df.rename(columns = {'Postcode':'PostalCode', 'Neighbourhood\n':'Neighbourhood'}, inplace = True) # rename the column Postcode to PostalCode and remove \n in Neighbourhood

In [10]:
df.head(12)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### Clean the dataframe by removing the non assigned boroughs. For neighbourhood which are not assigned but has the borough name, it would be replaced with the borough name.

In [11]:
df = df[df.Borough != 'Not assigned'] # remove boroughs which are not assigned

df['Neighbourhood'].replace("Not assigned", df['Borough'], inplace=True) # replace the name of neighbourhoods which are not assigned to borough name

In [12]:
df.head(12)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### The dataframe is refined by grouping the neighbourhoods with the same postal code and reseting the index number

In [13]:
df['Neighbourhood'] = df.groupby('PostalCode')['Neighbourhood'].transform(lambda neigh: ', '.join(neigh)) # Neighbourhood with the same postal code is to be grouped in the same row

df = df.drop_duplicates() # Any duplicates are dropped

if (df.index.name != 'PostalCode'): # before resetting the index number, the index is to be reassigned to postal code first
    df = df.set_index('PostalCode')
    
df.reset_index(inplace=True) # reset index creates new column

### Print the final verison of the dataframe

In [14]:
df.head(12)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### Print the number of rows. After cleaning the data, the rows have come down to 103 but column remains at 3

In [15]:
df.shape # print the number of rows and columns

(103, 3)

# End of part 1